# Sports14 Text/Image Feature Extraction

In [1]:

import os
import numpy as np
import pandas as pd

In [2]:
os.chdir('/home/exlysha/MMRec/mydata/Baby')
os.getcwd()

'/home/exlysha/MMRec/mydata/Baby'

## Load text data

In [3]:
i_id, desc_str = 'itemID', 'description'

file_path = './'
file_name = 'meta-sports14.csv'

meta_file = os.path.join(file_path, file_name)

df = pd.read_csv(meta_file)
df.sort_values(by=[i_id], inplace=True)

print('data loaded!')
print(f'shape: {df.shape}')

df[:3]

data loaded!
shape: (7050, 11)


,itemID,asin,categories,description,title,price,imUrl,brand,related,salesRank,results
0,0,097293751X,[['Baby']],Easily keep track of your baby's or child's da...,"Baby Tracker&reg; - Daily Childcare Journal, S...",17.00,http://ecx.images-amazon.com/images/I/41Bb6wf%...,Time Too,"{'also_bought': ['9729375011', 'B004FN1AE8', '...",NaN,Baby Tracker&reg; helps track your baby's dail...
1,1,9729375011,[['Baby']],This is version of the award-winningBaby Track...,Newborn Baby Tracker&reg; - Round the Clock Ch...,15.95,http://ecx.images-amazon.com/images/I/51r3BLpL...,,"{'also_bought': ['B000V5KPZ4', 'B001F8TLLU', '...",NaN,Newborn Baby Tracker&reg; - A round-the-clock ...
2,2,B00000IZQI,[['Baby']],This colorful car collection develops motor sk...,Fisher Price Nesting Action Vehicles,8.37,http://ecx.images-amazon.com/images/I/51E83QCC...,,"{'also_bought': ['B0042D69W4', 'B00428LIZM', '...",NaN,Fisher Price Nesting Action Vehicles help deve...


In [4]:
# sentences: title + brand + category + description | All have title + description

title_na_df = df[df['title'].isnull()]
print(title_na_df.shape)

desc_na_df = df[df['description'].isnull()]
print(desc_na_df.shape)

na_df = df[df['description'].isnull() & df['title'].isnull()]
print(na_df.shape)

na3_df = df[df['description'].isnull() & df['title'].isnull() & df['brand'].isnull()]
print(na3_df.shape)

na4_df = df[df['description'].isnull() & df['title'].isnull() & df['brand'].isnull() & df['categories'].isnull()]
print(na4_df.shape)

df[desc_str] = df[desc_str].fillna(" ")
df['title'] = df['title'].fillna(" ")
df['brand'] = df['brand'].fillna(" ")
df['categories'] = df['categories'].fillna(" ")

sentences = []
for i, row in df.iterrows():
    sen = row['title'] + ' ' + row['brand'] + ' '
    cates = eval(row['categories'])
    if isinstance(cates, list):
        for c in cates[0]:
            sen = sen + c + ' '
    sen += row[desc_str]
    sen = sen.replace('\n', ' ')

    sentences.append(sen)

sentences[:10]

(0, 11)
(0, 11)
(0, 11)
(0, 11)
(0, 11)


["Baby Tracker&reg; - Daily Childcare Journal, Schedule Log Time Too Baby Easily keep track of your baby's or child's daily schedules, activities and needs in one handy spot.  And make the drop-off and pick-up process for childcare a snap.  The Baby Tracker Daily Childcare Journal is designed to make it fast and easy to record and review meals, naps, activities, playtime, daily news, milestones and to-dos ---all in a single page view.  Get ready for doctor visits, give to childcare helpers and use this easy at-a-glance daily record to monitor schedules and prep for your baby's daily needs.  This version records daily activities from 6 a.m. to 7 p.m. and creates a visual snapshot of activities by time each day.   The Baby Tracker Daily Childcare Journal is ready to keep track of up to 6 months of daily schedule tracking (180 days) and is spiral bound to make it portable and lay flat for easy recording on a countertop or nightstand.Each journal includes:*180 daily at-a-glance tracking re

In [5]:
results = sentences

results[:10]

["Baby Tracker&reg; - Daily Childcare Journal, Schedule Log Time Too Baby Easily keep track of your baby's or child's daily schedules, activities and needs in one handy spot.  And make the drop-off and pick-up process for childcare a snap.  The Baby Tracker Daily Childcare Journal is designed to make it fast and easy to record and review meals, naps, activities, playtime, daily news, milestones and to-dos ---all in a single page view.  Get ready for doctor visits, give to childcare helpers and use this easy at-a-glance daily record to monitor schedules and prep for your baby's daily needs.  This version records daily activities from 6 a.m. to 7 p.m. and creates a visual snapshot of activities by time each day.   The Baby Tracker Daily Childcare Journal is ready to keep track of up to 6 months of daily schedule tracking (180 days) and is spiral bound to make it portable and lay flat for easy recording on a countertop or nightstand.Each journal includes:*180 daily at-a-glance tracking re

In [6]:
import clip
import torch
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [7]:

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14@336px", device=device)
sentence_embeddings = np.zeros((len(results), 768), dtype=np.float32)
for split in range(0, len(results), 1000):
    print(f'processing {split} to {split + 1000}')
    batch = results[split:split + 1000]
    text = clip.tokenize(batch, truncate=True).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text)
        text_features /= text_features.norm(dim=-1, keepdim=True)
    text_features = text_features.cpu().numpy()
    sentence_embeddings[split:split + 1000] = text_features
# text = clip.tokenize(results[0], truncate=True).to(device)
# with torch.no_grad():
#     text_features = model.encode_text(text)
#     text_features /= text_features.norm(dim=-1, keepdim=True)
# sentence_embeddings = text_features.cpu().numpy()
print('text encoded!')



100%|███████████████████████████████████████| 891M/891M [01:31<00:00, 10.2MiB/s]


processing 0 to 1000
processing 1000 to 2000
processing 2000 to 3000
processing 3000 to 4000
processing 4000 to 5000
processing 5000 to 6000
processing 6000 to 7000
processing 7000 to 8000
text encoded!


In [8]:
sentence_embeddings.shape,sentence_embeddings.dtype

((7050, 768), dtype('float32'))

In [9]:
assert sentence_embeddings.shape[0] == df.shape[0]
np.save(os.path.join(file_path, 'text_feat_vit_raw_336.npy'), sentence_embeddings)
print('done!')

done!


In [10]:
sentence_embeddings[:10]

array([[-0.00462723, -0.00045466,  0.01423645, ...,  0.02607727,
        -0.08026123, -0.02519226],
       [ 0.00028253, -0.02731323,  0.00524521, ...,  0.02816772,
        -0.06896973, -0.06915283],
       [-0.04885864, -0.08728027,  0.00906372, ...,  0.00049162,
        -0.01030731, -0.06039429],
       ...,
       [ 0.01409912,  0.03988647,  0.01638794, ..., -0.04449463,
        -0.00086451, -0.04089355],
       [-0.04171753, -0.05706787,  0.03851318, ...,  0.01649475,
        -0.03460693, -0.02737427],
       [-0.01831055, -0.05102539,  0.01812744, ...,  0.00581741,
        -0.05419922, -0.03756714]], dtype=float32)

In [11]:
load_txt_feat = np.load('text_feat_vit_raw_336.npy', allow_pickle=True)
print(load_txt_feat.shape)
load_txt_feat[:10]

(7050, 768)


array([[-0.00462723, -0.00045466,  0.01423645, ...,  0.02607727,
        -0.08026123, -0.02519226],
       [ 0.00028253, -0.02731323,  0.00524521, ...,  0.02816772,
        -0.06896973, -0.06915283],
       [-0.04885864, -0.08728027,  0.00906372, ...,  0.00049162,
        -0.01030731, -0.06039429],
       ...,
       [ 0.01409912,  0.03988647,  0.01638794, ..., -0.04449463,
        -0.00086451, -0.04089355],
       [-0.04171753, -0.05706787,  0.03851318, ...,  0.01649475,
        -0.03460693, -0.02737427],
       [-0.01831055, -0.05102539,  0.01812744, ...,  0.00581741,
        -0.05419922, -0.03756714]], dtype=float32)

# Image encoder (V0)，following LATTICE, averaging over for missed items

In [14]:
df[:5]

,itemID,asin,imUrl,description,categories,title,price,salesRank,related,brand
0,0,0528881469,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[['Electronics', 'GPS & Navigation', 'Vehicle ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",
1,1,0594451647,http://ecx.images-amazon.com/images/I/51RjSETO...,HDTV Adapter Kit for NOOK HD and NOOK HD+\nThi...,"[['Electronics', 'Computers & Accessories', 'T...",Barnes &amp; Noble HDTV Adapter Kit for NOOK H...,49.95,NaN,"{'also_bought': ['B009L7EEZA', 'B00AGAYQEU', '...",
2,2,0594481813,http://ecx.images-amazon.com/images/I/41K7ymN5...,Power up your device with this Barnes &amp; No...,"[['Electronics', 'eBook Readers & Accessories'...",Barnes &amp; Noble OV/HB-ADP Universal Power Kit,19.65,NaN,"{'also_bought': ['B00AAKLIIS', 'B00A668GUO', '...",Barnes &amp; Noble
3,3,0972683275,http://ecx.images-amazon.com/images/I/41hYJ9Mw...,The VideoSecu TV mount is a mounting solution ...,"[['Electronics', 'Accessories & Supplies', 'Au...",VideoSecu 24&quot; Long Arm TV Wall Mount Low ...,29.99,{},"{'also_bought': ['B000X3KOD2', 'B0074FGR74', '...",VideoSecu
4,4,1400501466,http://ecx.images-amazon.com/images/I/41StLxJb...,"Enjoy your digital books, magazines and newspa...","[['Electronics', 'Computers & Accessories', 'T...","Barnes &amp; Noble NOOK Tablet 16gb (Color, BN...",188.88,NaN,"{'also_bought': ['B007H564NS', 'B005HM8BRU', '...",


In [15]:
df.shape

(63001, 10)

In [16]:
import pandas as pd
import os
import requests
from urllib.parse import urlparse

save_dir = 'downloaded_images'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

invalid_urls = []

# 下载函数
def download_image(url, save_path):
    try:
        # 发送请求
        response = requests.get(url, timeout=10)
        # 检查请求是否成功
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded: {save_path}")
        else:
            invalid_urls.append(url)
            print(f"Failed to download: {url} (Status code: {response.status_code})")
    except Exception as e:
        invalid_urls.append(url)
        print(f"Error downloading {url}: {e}")

# 遍历 imUrl 列并下载
for index, row in df.iterrows():
    url = row['imUrl']
    if pd.isna(url) or not url.strip():  # 检查空或无效 URL
        invalid_urls.append(url)
        print(f"Skipping invalid URL at index {index}")
        continue
    
    # 从 URL 中提取文件名
    parsed_url = urlparse(url)
    file_name = f"{index}.jpg"
    
    # 构建保存路径
    save_path = os.path.join(save_dir, file_name)
    
    # 下载图像
    download_image(url, save_path)

Downloaded: downloaded_images/0.jpg
Downloaded: downloaded_images/1.jpg
Downloaded: downloaded_images/2.jpg
Downloaded: downloaded_images/3.jpg
Downloaded: downloaded_images/4.jpg
Downloaded: downloaded_images/5.jpg
Downloaded: downloaded_images/6.jpg
Downloaded: downloaded_images/7.jpg
Downloaded: downloaded_images/8.jpg
Downloaded: downloaded_images/9.jpg
Downloaded: downloaded_images/10.jpg
Downloaded: downloaded_images/11.jpg
Downloaded: downloaded_images/12.jpg
Downloaded: downloaded_images/13.jpg
Downloaded: downloaded_images/14.jpg
Downloaded: downloaded_images/15.jpg
Downloaded: downloaded_images/16.jpg
Downloaded: downloaded_images/17.jpg
Downloaded: downloaded_images/18.jpg
Downloaded: downloaded_images/19.jpg
Downloaded: downloaded_images/20.jpg
Downloaded: downloaded_images/21.jpg
Downloaded: downloaded_images/22.jpg
Downloaded: downloaded_images/23.jpg
Downloaded: downloaded_images/24.jpg
Downloaded: downloaded_images/25.jpg
Downloaded: downloaded_images/26.jpg
Downloaded:

In [17]:
invalid_urls

['http://ecx.images-amazon.com/images/I/51RC63MR1YL._SX300_.jpg',
 'http://ecx.images-amazon.com/images/I/51Xy1ZADlyL._SX300_.jpg',
 'http://ecx.images-amazon.com/images/I/11RK4NGAVVL.jpg',
 'http://ecx.images-amazon.com/images/I/51Q9046TTHL._SX300_.jpg',
 nan,
 'http://ecx.images-amazon.com/images/I/41SKJVAYTPL._SX300_.jpg',
 'http://ecx.images-amazon.com/images/I/51bBfpLLhOL._SX300_.jpg',
 nan,
 'http://ecx.images-amazon.com/images/I/51SRQX6GEFL._SY300_.jpg',
 nan,
 'http://ecx.images-amazon.com/images/I/51qXIFPm76L._SX300_.jpg',
 nan,
 nan,
 'http://ecx.images-amazon.com/images/I/41ld-pBPfML._SY300_.jpg',
 'http://ecx.images-amazon.com/images/I/31i7l71j-EL._SX300_.jpg',
 'http://ecx.images-amazon.com/images/I/51FFKaHMpuL._SY300_.jpg',
 'http://ecx.images-amazon.com/images/I/41LHVevSHrL._SX300_.jpg',
 'http://ecx.images-amazon.com/images/I/41EpJXN4ctL._SX300_.jpg',
 'http://ecx.images-amazon.com/images/I/41%2BLcxAJL4L._SY300_.jpg',
 'http://ecx.images-amazon.com/images/I/41%2BE2SF%2B

In [18]:
path='downloaded_images'
import os
currect=os.listdir(path)
currect=list(map(lambda x:x.split('.')[0],currect))


In [19]:
currect=list(map(int,currect))
currect.sort()
# currect[-1]
lost=[]
for i in range(0,currect[-1]+1):
    if i not in currect:
        lost.append(i)
lost

[153,
 1690,
 1734,
 3469,
 3550,
 3805,
 5409,
 5636,
 6106,
 8391,
 8883,
 9071,
 9722,
 10340,
 10395,
 10397,
 10500,
 10777,
 10804,
 10873,
 11392,
 11491,
 11567,
 11572,
 11733,
 11857,
 12322,
 16489,
 17136,
 17137,
 17138,
 17139,
 17157,
 17987,
 18133,
 18380,
 18383,
 18384,
 18385,
 19769,
 20105,
 20201,
 20917,
 21267,
 22741,
 23422,
 24406,
 24671,
 26863,
 28322,
 28383,
 28460,
 28826,
 28852,
 29381,
 29920,
 30019,
 30360,
 32099,
 32445,
 32607,
 33410,
 34637,
 34762,
 35176,
 37636,
 41297,
 42501,
 42953,
 43225,
 43553,
 45300,
 45346,
 46003,
 48589,
 48984,
 51248,
 54719,
 55029,
 55331,
 55985,
 60137,
 61787,
 61847,
 62576,
 62688]

In [20]:
assert len(lost)==len(invalid_urls)

In [21]:
new_lost = []

# 下载函数
def download_image(url, save_path):
    try:
        # 发送请求
        response = requests.get(url, timeout=10)
        # 检查请求是否成功
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded: {save_path}")
        else:
            new_lost.append(url)
            print(f"Failed to download: {url} (Status code: {response.status_code})")
    except Exception as e:
        new_lost.append(url)
        print(f"Error downloading {url}: {e}")

In [22]:
for i in range(len(lost)):
    url = invalid_urls[i]
    if pd.isna(url) or not url.strip():  # 检查空或无效 URL
        new_lost.append(url)
        print(f"Skipping invalid URL at index {i}")
        continue
    
    # 从 URL 中提取文件名
    parsed_url = urlparse(url)
    file_name = f"{lost[i]}.jpg"
    
    # 构建保存路径
    save_path = os.path.join(save_dir, file_name)
    
    # 下载图像
    download_image(url, save_path)

Failed to download: http://ecx.images-amazon.com/images/I/51RC63MR1YL._SX300_.jpg (Status code: 404)
Failed to download: http://ecx.images-amazon.com/images/I/51Xy1ZADlyL._SX300_.jpg (Status code: 404)
Failed to download: http://ecx.images-amazon.com/images/I/11RK4NGAVVL.jpg (Status code: 404)
Failed to download: http://ecx.images-amazon.com/images/I/51Q9046TTHL._SX300_.jpg (Status code: 404)
Skipping invalid URL at index 4
Downloaded: downloaded_images/3805.jpg
Failed to download: http://ecx.images-amazon.com/images/I/51bBfpLLhOL._SX300_.jpg (Status code: 404)
Skipping invalid URL at index 7
Failed to download: http://ecx.images-amazon.com/images/I/51SRQX6GEFL._SY300_.jpg (Status code: 404)
Skipping invalid URL at index 9
Failed to download: http://ecx.images-amazon.com/images/I/51qXIFPm76L._SX300_.jpg (Status code: 404)
Skipping invalid URL at index 11
Skipping invalid URL at index 12
Failed to download: http://ecx.images-amazon.com/images/I/41ld-pBPfML._SY300_.jpg (Status code: 404)

In [23]:
new_lost

['http://ecx.images-amazon.com/images/I/51RC63MR1YL._SX300_.jpg',
 'http://ecx.images-amazon.com/images/I/51Xy1ZADlyL._SX300_.jpg',
 'http://ecx.images-amazon.com/images/I/11RK4NGAVVL.jpg',
 'http://ecx.images-amazon.com/images/I/51Q9046TTHL._SX300_.jpg',
 nan,
 'http://ecx.images-amazon.com/images/I/51bBfpLLhOL._SX300_.jpg',
 nan,
 'http://ecx.images-amazon.com/images/I/51SRQX6GEFL._SY300_.jpg',
 nan,
 'http://ecx.images-amazon.com/images/I/51qXIFPm76L._SX300_.jpg',
 nan,
 nan,
 'http://ecx.images-amazon.com/images/I/41ld-pBPfML._SY300_.jpg',
 'http://ecx.images-amazon.com/images/I/31i7l71j-EL._SX300_.jpg',
 'http://ecx.images-amazon.com/images/I/51FFKaHMpuL._SY300_.jpg',
 'http://ecx.images-amazon.com/images/I/41LHVevSHrL._SX300_.jpg',
 'http://ecx.images-amazon.com/images/I/41EpJXN4ctL._SX300_.jpg',
 'http://ecx.images-amazon.com/images/I/41%2BLcxAJL4L._SY300_.jpg',
 'http://ecx.images-amazon.com/images/I/41%2BE2SF%2BeuL._SY300_.jpg',
 'http://ecx.images-amazon.com/images/I/51MxmJmw